# Importing Packages & Dataset

In [4]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.despine()
sns.set_context('talk', font_scale = 1)
import statsmodels.formula.api as smf
import os

In [5]:
def InitiateData():
    '''
    Imports the data set and prepare it for EDA and modelling
    '''
    global data
    
    data = pd.read_excel('default of credit card clients.xls', header=1, index_col=0)

    col_names = ['Balance', 'Sex', 'Education', 'Marriage', 'Age',
                'Paid_Sep', 'Paid_Aug', 'Paid_Jul', 'Paid_Jun', 'Paid_May', 'Paid_Apr',
                'Bill_Sep', 'Bill_Aug', 'Bill_Jul', 'Bill_Jun', 'Bill_May', 'Bill_Apr',
                'PayAmt_Sep', 'PayAmt_Aug', 'PayAmt_Jul', 'PayAmt_Jun', 'PayAmt_May', 'PayAmt_Apr',
                'Default']
    data.columns = col_names
    data = data.astype(str).astype(int)
    
    return data.head()

InitiateData()

,Balance,Sex,Education,Marriage,Age,Paid_Sep,Paid_Aug,Paid_Jul,Paid_Jun,Paid_May,...,Bill_Jun,Bill_May,Bill_Apr,PayAmt_Sep,PayAmt_Aug,PayAmt_Jul,PayAmt_Jun,PayAmt_May,PayAmt_Apr,Default
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


# Cleaning Data

## Analysis of missing data

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 1: Label response and predictors
response =['Default']
predictors = [x for x in list(data.columns) if x not in response]

# Step 2: Split data set
y = data[response].copy()
X = data[predictors].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)

# Step 3: Prepare for scaling
scaler = StandardScaler().fit(X_train)

# Step 4: Scale the data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Boosting

In [7]:
%%time
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from mlxtend.regressor import StackingCVRegressor

from sklearn.ensemble import GradientBoostingClassifier

#Basic Boosting
model = GradientBoostingClassifier()

tuning_parameters = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators' : [250, 500, 750, 1000, 1500],
    'max_depth' : [2 ,3, 4],
    'subsample' : [0.6, 0.8, 1.0]
}

# Using GridSearchCV would be too slow. Increase the number of iterations to explore more hyperparameter combinations.
gb = RandomizedSearchCV(model, tuning_parameters, n_iter = 20, cv = 10, return_train_score=False, n_jobs=4, random_state = 20)

gb.fit(X_train, y_train)


gb = gb.best_estimator_

Wall time: 4min 16s


In [22]:
stats.describe(gb.predict(X_train))

DescribeResult(nobs=3000, minmax=(0.00312916857941082, 0.9847440468334795), mean=0.22539070559329089, variance=0.03911565682341456, skewness=1.5946358972376458, kurtosis=1.795751873769099)

In [23]:
%%time
#XGBoost
import xgboost as xgb
model = xgb.XGBClassifier()


tuning_parameters = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators' : [250, 500, 750, 1000, 1500],
    'max_depth' : [2, 3, 4],
    'subsample' : [0.6, 0.8, 1.0],
}

gb_search = RandomizedSearchCV(model, tuning_parameters, n_iter = 16, cv = 5, return_train_score=False, n_jobs=4,
                              random_state = 20)
gb_search.fit(X_train, y_train)

xbst = gb_search.best_estimator_

Wall time: 1min 12s


In [24]:
stats.describe(xbst.predict(X_train))

DescribeResult(nobs=3000, minmax=(0, 1), mean=0.11833333333333333, variance=0.10436534400355676, skewness=2.363243475881673, kurtosis=3.584919726297291)

In [25]:
%%time
import lightgbm as lgb
model = lgb.LGBMClassifier()


tuning_parameters = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators' : [250, 500, 750, 1000, 1500],
    'max_depth' : [2, 3, 4],
    'subsample' : [0.6, 0.8, 1.0],
}

gb_search = RandomizedSearchCV(model, tuning_parameters, n_iter = 16, cv = 5, return_train_score=False, n_jobs=4, 
                               random_state = 20)

gb_search.fit(X_train, y_train)

lbst = gb_search.best_estimator_

Wall time: 56.8 s


In [70]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

lasso = Pipeline((
    ('scaler', StandardScaler()),
    ('estimator', LassoCV(cv=5)),
))

lasso.fit(X_train, y_train)

y_fit = lasso.predict(X_train)
resid = y_train['Default'].values - y_fit

In [26]:
stats.describe(lbst.predict(X_train))

DescribeResult(nobs=3000, minmax=(0, 1), mean=0.115, variance=0.10180893631210405, skewness=2.413627079052996, kurtosis=3.825595676737901)

# SVC

In [38]:
from sklearn.svm import SVC
model = SVC(gamma='auto')
tuning_parameters = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

clf = RandomizedSearchCV(model, tuning_parameters, n_iter = 4, cv = 10, return_train_score=False, n_jobs=4, random_state = 20)

clf.fit(X_train, y_train) 

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=4, n_jobs=4,
          param_distributions={'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
          pre_dispatch='2*n_jobs', random_state=20, refit=True,
          return_train_score=False, scoring=None, verbose=0)

In [40]:
stats.describe(clf.predict(X_train))

DescribeResult(nobs=3000, minmax=(0, 1), mean=0.10366666666666667, variance=0.09295087251305992, skewness=2.600376632074569, kurtosis=4.7619586286394835)

# Neural Net

In [96]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

NN = Sequential()

# Layers
NN.add(Dense(output_dim = 10, init = 'uniform', activation = 'relu'))
NN.add(Dense(output_dim = 10, init = 'uniform', activation = 'relu'))
NN.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling Neural Network
NN.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


NN.fit(X_train, y_train['Default'].values, batch_size = 10, epochs=10)

Epoch 1/10
3000/3000 [==============================] - 1s 324us/step - loss: 0.5661 - acc: 0.7737
Epoch 2/10
3000/3000 [==============================] - 0s 138us/step - loss: 0.5009 - acc: 0.7747
Epoch 3/10
3000/3000 [==============================] - 0s 128us/step - loss: 0.4902 - acc: 0.7747
Epoch 4/10
3000/3000 [==============================] - 0s 127us/step - loss: 0.4830 - acc: 0.7747
Epoch 5/10
3000/3000 [==============================] - 1s 171us/step - loss: 0.4771 - acc: 0.7773
Epoch 6/10
3000/3000 [==============================] - 0s 139us/step - loss: 0.4731 - acc: 0.8013
Epoch 7/10
3000/3000 [==============================] - 1s 182us/step - loss: 0.4705 - acc: 0.8083
Epoch 8/10
3000/3000 [==============================] - 0s 165us/step - loss: 0.4680 - acc: 0.8117
Epoch 9/10
3000/3000 [==============================] - 1s 169us/step - loss: 0.4651 - acc: 0.8110
Epoch 10/10
3000/3000 [==============================] - 0s 120us/step - loss: 0.4635 - acc: 0.8143


In [97]:
NN.predict(X_test)

array([[0.224285  ],
       [0.17293058],
       [0.21888234],
       ...,
       [0.63648164],
       [0.06232405],
       [0.5088713 ]], dtype=float32)

# Model Stacking

In [ ]:
# Idk how to put logistic regression as the meta_classifier 
    # Been trying to find that out but haven't found out that yet
    # Could be "lr" but idk if that means logistic regression
    # I believe it is based on this (https://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier/#example-3-stacked-classification-and-gridsearch)
        # Look at this example's output 

#%%time
#
#from mlxtend.classifier import StackingClassifier
#
#models = [tree1, tree2, bag, rf]
#
#stack = StackingClassifier(models, meta_classifier=lr, cv=10)
#stack.fit(X_train.values, y_train.ravel())

# Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score

columns = ['Error rate', 'True Positive Rate', 'True Negative Rate', 'AUC', 'Precision']
rows = ['Decision Tree 1', 'Decision Tree 2', 'Bagged Trees', 'Random Forest', 'KNN']
results = pd.DataFrame(0.0, columns=columns, index=rows) 

methods = [tree1, tree2, bag, rf, knn]

y_prob = np.zeros((len(test), len(rows)))

for i, method in enumerate(methods):
    
    y_pred = method.predict(X_test)
    y_prob[:, i] = method.predict_proba(X_test)[:,1]

    confusion = confusion_matrix(y_test, y_pred) 

    results.iloc[i,0] = 1 - accuracy_score(y_test, y_pred)
    results.iloc[i,1] = confusion[1,1]/np.sum(confusion[1,:])
    results.iloc[i,2] = confusion[0,0]/np.sum(confusion[0,:])
    results.iloc[i,3] = roc_auc_score(y_test, y_prob[:,i])
    results.iloc[i,4] = precision_score(y_test, y_pred)

results.round(3)

In [ ]:
# Plot ROC curves
from statlearning import plot_roc_curves

with sns.color_palette(crayon):
    fig, ax = plot_roc_curves(y_test, y_prob, labels=pd.Series(rows))
    plt.show()